In [54]:
import tensorflow as tf
import numpy as np
import functools
from tqdm import tqdm 
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras import metrics, losses
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('./data/bubble_detection.csv')

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
features = scaler.fit_transform(df.iloc[:,1:-1])
output = np.array(df.iloc[:,-1], dtype='float64')

In [9]:
time_steps = 36
x = []
y = []
for row in range(len(features) - time_steps):
    x.append(features[row : row + time_steps])
    y.append(output[row + time_steps - 1])

In [10]:
x = np.array(x, dtype = 'float64')
y = np.array(y, dtype = 'float64')

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=202144)

In [12]:
1 - sum(y_train) / len(y_train)

0.9460431654676259

In [13]:
model = Sequential([
    Bidirectional(LSTM(units=50,
                    input_shape=(x_train.shape[1], x_train.shape[2]),
                    recurrent_activation='sigmoid',
                    recurrent_initializer='glorot_uniform')),
    Dense(units=1, activation='sigmoid')
])

In [20]:
model.compile(optimizer=Adam(learning_rate=1e-2),
            loss=binary_focal_loss(alpha=0.75),
            metrics=[metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall()])

In [21]:
model.fit(x_train, y_train)

Train on 556 samples
556/556 [==============================] - 5s 8ms/sample - loss: 0.0394 - binary_accuracy: 0.9424 - precision: 0.2500 - recall: 0.0333


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional multiple                  25200     
_________________________________________________________________
dense (Dense)                multiple                  101       
Total params: 25,301
Trainable params: 25,301
Non-trainable params: 0
_________________________________________________________________


In [23]:
yhat = model.predict_classes(x_test)

In [57]:
prob_output = pd.DataFrame(data = {'p': model.predict(x_test).flatten(), 'reference': y_test})
prob_output.to_csv('rnn_prob_output.csv')

In [58]:
len(x_train)

556

In [25]:
sklearn.metrics.confusion_matrix(y_test, yhat)

array([[134,   0],
       [  6,   0]])

In [19]:
# imported from: https://github.com/aldi-dimara/keras-focal-loss/blob/master/focal_loss.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 19 08:20:58 2018
@OS: Ubuntu 18.04
@IDE: Spyder3
@author: Aldi Faizal Dimara (Steam ID: phenomos)
"""

import tensorflow.keras.backend as K
import tensorflow as tf

def binary_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha_t*((1-p_t)^gamma)*log(p_t)
        
        p_t = y_pred, if y_true = 1
        p_t = 1-y_pred, otherwise
        
        alpha_t = alpha, if y_true=1
        alpha_t = 1-alpha, otherwise
        
        cross_entropy = -log(p_t)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1-y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true)*alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1-alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1-p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss